# Clase Práctica 2: Support Vector Machine (SVM) y Metricas


----------------------------------

## Objetivos de la clase 📚

El objetivo de esta clase práctica es proporcionar una comprensión básica de los modelos de Support Vector Machine (SVM) y cómo evaluar su rendimiento utilizando métricas en la biblioteca scikit-learn de Python. Los SVM son poderosos algoritmos de aprendizaje supervisado utilizados para problemas de clasificación y regresión. Para esto, implementaremos modelos SVM de clasificación con diferentes hiperparametros para poder relacionar las características fisiológicas de pacientes con la presencia de una enfermedad cardiovascular 🩺. 


## **Clasificación**

**Exploración de datos 🧐**

**Importar las librerías**

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

**Lectura del dataset**

Con la función `read_csv` podemos cargar datasets que se encuentren en formato csv. Es importante que el nombre de la variable sea lo más simple posible, ya que será utilizado frecuentemente.

In [8]:
df = pd.read_csv("https://raw.githubusercontent.com/fvillena/biocompu/2022/data/cardiovascular_diseases.csv")

La función `head` nos muestra las primeras 5 filas del dataset

In [9]:
df.head()

,age,male,chest_pain_type,resting_blood_pressure,cholesterol,high_fasting_blood_sugar,resting_electrocardiographic_results,maximum_heart_rate_achieved,exercise_induced_angina,st_depression_by_exercise,slope_st_by_excercise_peak,number_of_vessels_colored_by_flouroscopy,thalassemia,cardiovascular_disease
0,45,0,typical_angina,138,236,0,0,152,1,0.2,0,0,fixed_defect,1
1,61,1,typical_angina,120,260,0,1,140,1,3.6,0,1,reversable_defect,0
2,44,0,non_anginal_pain,108,141,0,1,175,0,0.6,0,0,fixed_defect,1
3,77,1,typical_angina,125,304,0,0,162,1,0.0,-1,3,fixed_defect,0
4,35,1,typical_angina,126,282,0,0,156,1,0.0,-1,0,reversable_defect,0


In [6]:
df.shape 

(301, 14)

El resultado de la celda anterior nos muestra que hay un total de 301 instancias, donde cada una tiene un total de 14 atributos. Para acceder al nombre de cada uno de estos atributos, utilizamos el comando `columns`.

In [7]:
df.columns

Index(['age', 'male', 'chest_pain_type', 'resting_blood_pressure',
       'cholesterol', 'high_fasting_blood_sugar',
       'resting_electrocardiographic_results', 'maximum_heart_rate_achieved',
       'exercise_induced_angina', 'st_depression_by_exercise',
       'slope_st_by_excercise_peak',
       'number_of_vessels_colored_by_flouroscopy', 'thalassemia',
       'cardiovascular_disease'],
      dtype='object')

Luego, podemos utilizar la función `sum`, que nos permite sumar los valores que son `True` en cada columna.

In [11]:
df.isnull().sum()

age                                         0
male                                        0
chest_pain_type                             0
resting_blood_pressure                      0
cholesterol                                 0
high_fasting_blood_sugar                    0
resting_electrocardiographic_results        0
maximum_heart_rate_achieved                 0
exercise_induced_angina                     0
st_depression_by_exercise                   0
slope_st_by_excercise_peak                  0
number_of_vessels_colored_by_flouroscopy    0
thalassemia                                 0
cardiovascular_disease                      0
dtype: int64

En este caso no tenemos valores nulos, pero si los tuviéramos es importante que sean reemplazados por algún valor. 

Ahora vamos a analizar nuestro **atributo objetivo**. Este atributo corresponde al valor que nos gustaría predecir a partir de los demás atributos (o parte de ellos). En nuestro caso utilizaremos la columna  *cardiovascular_disease*. Accedamos a ella para ver la naturaleza de los datos.

In [12]:
df["cardiovascular_disease"]

0      1
1      0
2      1
3      0
4      0
      ..
296    0
297    1
298    1
299    0
300    0
Name: cardiovascular_disease, Length: 301, dtype: int64

Con este resultados podemos concluir que trabajaremos con una clasificación binaria 👀. Otro paso fundamental, es contar la frecuencia de cada uno de los posibles valores pertenecientes a nuestro atributo/variable objetivo. Esto se realiza con la función `value_counts`.

### ¿Cuando un dataset esta desbalanceado?

Un dataset se considera desbalanceado para un problema de clasificación cuando una o varias de las clases están representadas de manera significativamente desproporcionada en comparación con las demás clases. Es decir, cuando la cantidad de ejemplos de una clase es mucho mayor o mucho menor que la cantidad de ejemplos de las otras clases en el conjunto de datos.

En un problema de clasificación binaria, un dataset está desbalanceado si una de las clases es dominante y representa la gran mayoría de las muestras, mientras que la otra clase es una minoría significativa. Por ejemplo, si en un conjunto de datos de detección de fraude de tarjetas de crédito, el 99% de las transacciones son no fraudulentas (clase negativa) y solo el 1% son fraudulentas (clase positiva), entonces el dataset está desbalanceado.

En el caso de problemas de clasificación multiclase, un dataset está desbalanceado si algunas clases están sobrerepresentadas y otras están subrepresentadas en comparación con la cantidad total de muestras. Por ejemplo, si en un dataset de reconocimiento de especies de flores, una especie representa el 80% de las muestras mientras que las otras especies se dividen en el 20% restante, entonces el dataset está desbalanceado.

In [13]:
df["cardiovascular_disease"].value_counts()

cardiovascular_disease
1    164
0    137
Name: count, dtype: int64

Podemos ver que los datos están prácticamente balanceados; 164 pacientes presentan una enfermedad cardiovascular, mientras que 137 no tienen. En caso que exista un desbalance, hay que aplicar técnicas especiales de balanceo que veremos más adelante en el curso. Ahora sigamos analizando nuestros datos viendo un ejemplo.

In [14]:
df.sample(1)

,age,male,chest_pain_type,resting_blood_pressure,cholesterol,high_fasting_blood_sugar,resting_electrocardiographic_results,maximum_heart_rate_achieved,exercise_induced_angina,st_depression_by_exercise,slope_st_by_excercise_peak,number_of_vessels_colored_by_flouroscopy,thalassemia,cardiovascular_disease
183,52,1,typical_angina,125,212,0,1,168,0,1.0,-1,2,reversable_defect,0


Podemos ver que las columnas *chest_pain_type* y *thalassemia* son atributos con valores de tipo string (palabras). En aprendizaje de máquinas nuestros algoritmos o sistemas están basados en una serie de operaciones matemáticas sobre los datos, lo cual nos permite entrenar y luego realizar predicciones. Debido a esto, los modelos no pueden trabajar directamente con texto y estos deben ser transformados a números.

La manera más sencilla de realizar esto es utilizando una transformación **One-Hot**. Esto significa que si tenemos un atributo de tipo string, el cual tiene `n` valores distintos, crearemos un total de `n` columnas nuevas con valores numéricos y binarios. Para ejemplificar, si se tiene una columna con los siguientes valores


| chest_pain_type |
|--------|
| asymptomatic |
| non_anginal_pain   |
| typical_angina|
| atypical_angina |

y se transforma utilizando One-Hot Encoding, se generan las siguientes asignaciones columnas:


| chest_pain_type_asymptomatic | chest_pain_type_non_anginal_pain | chest_pain_type_typical_angina | chest_pain_type_atypical_angina |
|------------------------------|----------------------------------|--------------------------------|---------------------------------|
| 1                            | 0                                | 0                              | 0                               |
| 0                            | 1                                | 0                              | 0                               |
| 0                            | 0                                | 1                              | 0                               |
| 0                            | 0                                | 0                              | 1                               |


Así, ya no tenemos que lidiar con valores no numéricos, y la cantidad de características aumenta. Para realizar esta transformación podemos utilizar la función `get_dummies`.

In [15]:
df = pd.get_dummies(df)
df.sample(3)

,age,male,resting_blood_pressure,cholesterol,high_fasting_blood_sugar,resting_electrocardiographic_results,maximum_heart_rate_achieved,exercise_induced_angina,st_depression_by_exercise,slope_st_by_excercise_peak,number_of_vessels_colored_by_flouroscopy,cardiovascular_disease,chest_pain_type_asymptomatic,chest_pain_type_atypical_angina,chest_pain_type_non_anginal_pain,chest_pain_type_typical_angina,thalassemia_fixed_defect,thalassemia_normal,thalassemia_reversable_defect
277,41,1,130,214,0,0,168,0,2.0,0,0,1,False,False,True,False,True,False,False
48,41,1,112,250,0,1,179,0,0.0,-1,0,1,False,False,True,False,True,False,False
80,34,1,118,182,0,0,174,0,0.0,-1,0,1,True,False,False,False,True,False,False


**Entrenamiento de modelos**

Para poder entrenar nuestros modelos de aprendizaje supervisado debemos separar nuestros datos identificando claramente cuál será nuestra variable objetivo, es decir, la que queremos predecir. 

Generalmente se utiliza la variable `X` para los atributos de entrada o features, mientras que la variable `y` es el objetivo.

In [16]:
X = df.drop("cardiovascular_disease", axis=1) # Drop elimina la columna que se especifique, y axis = 1 significa eliminar valores de las columnas

In [17]:
y = df['cardiovascular_disease']

**Support Vector Classification (SVC)**



Una explicación muy general sobre este modelo, es que representa a los puntos de muestra en el espacio, separando las clases a 2 espacios lo más amplios posibles mediante un hiperplano de separación. Cuando llega un nuevo ejemplo, se analiza a cuál de los dos espacios pertenece para realizar la clasificación. En general, este algoritmo funciona muy bien con datos que son linealmente separables. En caso que no sea así, este algoritmo puede trasladar el problema de un problema no separable linealmente a uno que sí lo es. Veamos como podemos programar un modelo SVC.

In [18]:
from sklearn.svm import SVC

Para evaluar el rendimiento de nuestro modelo utilizamos la función `accuracy_score`. Esta función nos permite obtener la proporción de ejemplos que fueron correctamente clasificados sobre el total.

In [19]:
from sklearn.metrics import accuracy_score
import numpy as np

In [20]:
from sklearn.model_selection import train_test_split

### ¿Cual es una buena proporcion para los datos de entrenamiento y testeo?

Los datos de entrenamiento son el conjunto de datos sobre los que se realiza el entrenamiento real. El conjunto de validación ayuda a mejorar el rendimiento del modelo afinándolo después de cada epoca. El conjunto de prueba nos informa de la precisión final del modelo tras completar la fase de entrenamiento.

El conjunto de entrenamiento no debe ser demasiado pequeño; de lo contrario, el modelo no tendrá suficientes datos para aprender. Por otro lado, si el conjunto de validación es demasiado pequeño, las métricas de evaluación como la exactitud, la precisión, la recuperación y la puntuación F1 tendrán una gran varianza y no conducirán a un ajuste adecuado del modelo.

En general, poner el 80% de los datos en el conjunto de entrenamiento, el 10% en el conjunto de validación y el 10% en el conjunto de prueba es una buena división para empezar.
La división óptima de los conjuntos de prueba, validación y entrenamiento depende de factores como el caso de uso, la estructura del modelo, la dimensión de los datos, etc.

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=12)

Luego, creamos y entrenamos nuestro modelo sobre el conjunto de entrenamiento.

In [22]:
svc_model = SVC()
svc_model.fit(X_train, Y_train)

SVC()

Obtenemos las predicciones sobre el conjunto de test y calculamos el rendimiento.

In [23]:
predictions = svc_model.predict(X_test)

In [24]:
print(f'El accuracy obtenido por el modelo SVC es: {np.round(accuracy_score(Y_test, predictions), 2)}')

El accuracy obtenido por el modelo SVC es: 0.67


In [25]:
# Podemos obtener los indices de los datos que nos permiten crear el hiperplano
support_vector_indices = svc_model.support_
print(support_vector_indices)

[  1   2   5   7  11  12  13  14  15  19  21  22  23  24  25  26  28  30
  32  35  38  42  43  45  54  56  57  59  62  65  68  73  79  83  85  86
  87  89  92  96  98 102 106 107 108 110 112 117 118 122 127 131 135 137
 143 147 148 149 152 153 154 155 156 157 160 163 164 166 167 170 171 172
 173 174 175 176 177 180 183 184 186 190 191 192 194 195 199 200 201 203
 205 208   0   3   4   6   8   9  16  17  20  27  31  33  36  37  44  46
  47  48  49  51  52  53  55  60  61  63  66  67  69  70  72  75  76  77
  80  82  84  88  90  91  93  94  95  97  99 101 105 109 111 113 114 115
 116 119 120 121 124 125 126 128 129 130 132 133 134 136 138 139 140 141
 144 145 146 150 158 161 162 165 168 169 179 182 185 187 189 193 196 197
 202 204 206 207 209]


Dado que estamos trabajando el múltiples dimensiones no es posible generar un gráfico adecuado, sin embargo, si tuvieramos un problema de clasificación binaria con sólo dos atributos de entrada, podríamos generar visualizaciones como la siguiente. Este método es diferente de una Regresión logistica, ya que busca encontrar la máxima separación entre los conjuntos, no una sóla recta.

![image info](https://github.com/christianversloot/machine-learning-articles/raw/main/images/support_vectors.png)

### Hiperparametros de SVM

1.- C (Regularización):
* Definición: El parámetro de regularización C controla el equilibrio entre maximizar el margen y minimizar el error de clasificación en el modelo SVM.

* Interpretación: Un valor más alto de C permite que el SVM clasifique correctamente más puntos de entrenamiento, incluso si el límite de decisión es más ajustado. Un valor más bajo de C da prioridad a un margen más grande, permitiendo que algunos puntos de entrenamiento se clasifiquen incorrectamente.

2.- kernel (Kernel Function):

* Definición: El kernel es una función que mapea los datos de entrada a un espacio de mayor dimensión para permitir la clasificación de datos no linealmente separables en el espacio original.
 
* Interpretación: Los kernels más comunes son 'linear' para hiperplanos lineales, 'rbf' (Radial Basis Function) para problemas no lineales y 'poly' para kernels polinómicos. Elegir el kernel adecuado depende de la naturaleza de los datos y si son o no linealmente separables.

3.- gamma:

* Definición: Parámetro utilizado en los kernels 'rbf' y 'poly' para controlar el alcance de influencia de un solo ejemplo de entrenamiento.

* Interpretación: Un valor bajo de gamma implica un alcance más amplio, lo que puede llevar a un modelo sobreajustado. Un valor alto de gamma tiene un alcance más limitado, lo que puede llevar a un modelo poco ajustado.

4.- degree:

* Definición: Parámetro específico del kernel polinómico que representa el grado del polinomio utilizado en la transformación de características.

* Interpretación: Un grado más alto implica una mayor complejidad en el modelo, ya que crea características polinómicas de mayor orden.

5.- coef0:

* Definición: Parámetro específico de los kernels 'poly' y 'sigmoid' que controla la influencia de los términos de grado más alto en la función del kernel.

* Interpretación: Un valor más alto de coef0 da más peso a los términos de grado más alto en la función del kernel.

6.- probability:

* Definición: Parámetro booleano que indica si el modelo debe habilitar la estimación de probabilidades para las predicciones.

* Interpretación: Si se establece en True, se puede utilizar el método predict_proba() para obtener estimaciones de probabilidades en lugar de solo las etiquetas de clase.

### Primera combinación

In [26]:
svc_model_1 = SVC( kernel='linear', C= 1.0)
svc_model_1.fit(X_train, Y_train)

SVC(kernel='linear')

In [27]:
predictions = svc_model_1.predict(X_test)

In [28]:
print(f'El accuracy obtenido por el modelo 1 de SVC es: {np.round(accuracy_score(Y_test, predictions), 2)}')

El accuracy obtenido por el modelo 1 de SVC es: 0.78


### Segunda combinación

In [29]:
svc_model_2 = SVC( kernel = 'rbf', C = 1.0, gamma = 'scale')
svc_model_2.fit(X_train, Y_train)

SVC()

In [30]:
predictions = svc_model_2.predict(X_test)

In [31]:
print(f'El accuracy obtenido por el modelo 2 de SVC es: {np.round(accuracy_score(Y_test, predictions), 2)}')

El accuracy obtenido por el modelo 2 de SVC es: 0.67


### Tercera combinación

In [32]:
svc_model_3 = SVC( kernel = 'poly', C = 1.0, degree = 3)
svc_model_3.fit(X_train, Y_train)

SVC(kernel='poly')

In [33]:
predictions = svc_model_3.predict(X_test)

In [34]:
print(f'El accuracy obtenido por el modelo 3 de SVC es: {np.round(accuracy_score(Y_test, predictions), 2)}')

El accuracy obtenido por el modelo 3 de SVC es: 0.68


## Metricas

En el aprendizaje automático, cuando trabajamos con modelos de clasificación, es fundamental evaluar el rendimiento de nuestros modelos para comprender cuán bien están haciendo sus predicciones. Existen varias métricas que nos ayudan a medir diferentes aspectos del rendimiento de un clasificador. Aquí, exploraremos cuatro métricas clave: accuracy (exactitud), precision (precisión), recall (recuperación o sensibilidad) y F1-score.

Accuracy (Exactitud):

* La accuracy es una métrica básica y comúnmente utilizada para evaluar el rendimiento general de un clasificador. Representa la proporción de predicciones correctas realizadas por el modelo en comparación con el número total de predicciones. Es la relación entre las predicciones correctas y el total de observaciones.

* Cuando usarlo: La accuracy es adecuada cuando las clases están equilibradas en el conjunto de datos, es decir, cuando la cantidad de ejemplos de cada clase es similar. Sin embargo, puede ser engañosa en conjuntos de datos desequilibrados, donde una clase tiene una frecuencia mucho mayor que otra, ya que el modelo podría ser preciso simplemente prediciendo siempre la clase mayoritaria.

Precision (Precisión):

* La precision mide la proporción de verdaderos positivos (instancias correctamente clasificadas como positivas) en relación con el total de predicciones positivas realizadas por el clasificador. Es una métrica útil para evaluar cuántos de los ejemplos clasificados como positivos realmente pertenecen a la clase positiva.

* Cuando usarlo: La precision es especialmente importante cuando el coste de los falsos positivos es alto, lo que significa que clasificar incorrectamente ejemplos negativos como positivos puede tener consecuencias significativas.

Recall (Recuperación o Sensibilidad):

* El recall mide la proporción de verdaderos positivos en relación con el total de ejemplos positivos presentes en el conjunto de datos. Es una métrica que nos dice cuántos de los ejemplos positivos el modelo es capaz de recuperar correctamente.

* Cuando usarlo: El recall es relevante cuando el coste de los falsos negativos es alto, lo que implica que clasificar incorrectamente ejemplos positivos como negativos puede tener consecuencias graves.

F1-score:

* El F1-score es una métrica que combina la precisión y el recall en un solo valor, calculando la media armónica entre ambos. Proporciona una medida ponderada del rendimiento del clasificador que tiene en cuenta tanto los falsos positivos como los falsos negativos.

* Cuando usarlo: El F1-score es útil cuando se necesita un equilibrio entre la precisión y el recall. Es especialmente relevante en problemas donde las clases están desequilibradas y queremos tener un rendimiento equilibrado en ambas métricas.

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

svm_configs = [
    {}, # por defecto
    {'kernel': 'linear', 'C': 1.0},
    {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'},
    {'kernel': 'poly', 'C': 1.0, 'degree': 3}
]

# Entrenar y evaluar cada versión de SVM
for i, config in enumerate(svm_configs):
    svm = SVC(**config)
    svm.fit(X_train,Y_train)
    y_pred = svm.predict(X_test)

    # Evaluar el rendimiento del modelo
    accuracy = accuracy_score(Y_test, y_pred)
    precision = precision_score(Y_test, y_pred)
    recall = recall_score(Y_test, y_pred)
    f1 = f1_score(Y_test, y_pred)

    print(f"Resultados del SVM {i+1}:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-score: {f1:.2f}")
    print("-------------------------")

Resultados del SVM 1:
Accuracy: 0.67
Precision: 0.62
Recall: 0.94
F1-score: 0.75
-------------------------
Resultados del SVM 2:
Accuracy: 0.78
Precision: 0.73
Recall: 0.91
F1-score: 0.81
-------------------------
Resultados del SVM 3:
Accuracy: 0.67
Precision: 0.62
Recall: 0.94
F1-score: 0.75
-------------------------
Resultados del SVM 4:
Accuracy: 0.68
Precision: 0.63
Recall: 0.94
F1-score: 0.75
-------------------------


In [37]:
from sklearn.metrics import classification_report

svm_configs = [
    {}, # por defecto
    {'kernel': 'linear', 'C': 1.0},
    {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'},
    {'kernel': 'poly', 'C': 1.0, 'degree': 3}
]

# Entrenar y evaluar cada versión de SVM
for i, config in enumerate(svm_configs):
    svm = SVC(**config)
    svm.fit(X_train,Y_train)
    y_pred = svm.predict(X_test)


    print(f"Resultados del SVM {i+1}:")
    print(classification_report(Y_test, y_pred))
    
    print("-------------------------")

Resultados del SVM 1:
              precision    recall  f1-score   support

           0       0.85      0.39      0.53        44
           1       0.62      0.94      0.75        47

    accuracy                           0.67        91
   macro avg       0.73      0.66      0.64        91
weighted avg       0.73      0.67      0.64        91

-------------------------
Resultados del SVM 2:
              precision    recall  f1-score   support

           0       0.88      0.64      0.74        44
           1       0.73      0.91      0.81        47

    accuracy                           0.78        91
   macro avg       0.80      0.78      0.77        91
weighted avg       0.80      0.78      0.78        91

-------------------------
Resultados del SVM 3:
              precision    recall  f1-score   support

           0       0.85      0.39      0.53        44
           1       0.62      0.94      0.75        47

    accuracy                           0.67        91
   macro a